In [ ]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import time
import numpy as np
import itertools
from tqdm import tqdm

In [ ]:
#サイトのURL
detail_url = 'https://tabelog.com/rstLst/1/?Srt=D&SrtT=rt&sort_mode=1'

In [ ]:
res = requests.get(detail_url)
res.encoding = res.apparent_encoding
soup = BeautifulSoup(res.text, 'html.parser')

In [ ]:
num = ["1","2","3","4","5"]
all_html = []
for i in num:
  # Responseオブジェクト生成
  res = requests.get('https://tabelog.com/rstLst/'+i+'/?Srt=D&SrtT=rt&sort_mode=1')
  # 文字化け防止
  res.encoding = res.apparent_encoding
  # BeautifulSoupオブジェクト生成
  soup = BeautifulSoup(res.text, 'html.parser')
  all_html.append(soup) #soupを空のリストに入れる
  time.sleep(1) #1秒休む


In [ ]:
tabe_1 = []
for i in all_html:
  soup = i.find_all( class_='list-rst__rst-name-target')
  tabe_1.append(soup)

In [ ]:
res = itertools.chain(*tabe_1)
tabe_2 = list(res)

In [ ]:
#お店のURLを取得
restaurant_url_list = [] #空のリスト作成
for tabe in tabe_2:
  tabe_url = tabe.get('href') #href属性だけを取り出す
  restaurant_url_list.append(tabe_url) #restaurant_url_listにtabe_urlを入れる

In [ ]:
#それぞれのページのhtmlを取得
tabe_soup_list = [] #空のリスト作成
for tabe_url in tqdm(restaurant_url_list):
  # Responseオブジェクト生成
  res = requests.get(tabe_url)
  # 文字化け防止
  res.encoding = res.apparent_encoding
  # BeautifulSoupオブジェクト生成
  soup = BeautifulSoup(res.text, 'html.parser')
  tabe_soup_list.append(soup) #soupを空のリストに入れる
  time.sleep(1) #1秒休む

In [ ]:
#店名をrestaurant_name_listに格納
restaurant_name_list = []#空のリストを作成
for tabe_soup in tabe_soup_list:
  tabe_name = tabe_soup.find('h2', class_="display-name").text#タグとクラスを指定する
  tabe_name = tabe_name.replace('\n','')
  restaurant_name_list.append(tabe_name)#tebe_nameをrestaurant_name_listに格納する

In [ ]:
#評価をrestaurant_hosi_listに格納
restaurant_hosi_list = []#空のリストを作成
for tabe_soup in tabe_soup_list:
  tabe_hosi = tabe_soup.find('span', class_='rdheader-rating__score-val-dtl').text
  restaurant_hosi_list.append(tabe_hosi)#tabe_hosiをrestaurant_hosi_listに格納する

In [ ]:
#評価数をrestaurant_eva_listに格納
restaurant_eva_list = []
for tabe_soup in tabe_soup_list:
  tabe_eva = tabe_soup.find('span', class_='rdheader-rating__hozon-target')
  tabe_eva = tabe_eva.find('em', class_='num').text
  restaurant_eva_list.append(tabe_eva)

In [ ]:
#それぞれのデータをくっつける
tuple_element_list = [ [name, hosi, eva] for name, hosi, eva in zip (restaurant_name_list, restaurant_hosi_list,restaurant_eva_list)]
taberogu1 = pd.DataFrame(tuple_element_list, columns=['名前', '星', '評価数'])

とりあえずここまで

In [ ]:
#最寄駅をrestaurant_nearest_stationに格納
restaurant_nearest_station_list = []
for tabe_soup in tabe_soup_list:
  tabe_station = tabe_soup.find('div', class_='linktree__parent').text
  tabe_station = tabe_station.replace('\n','')
  restaurant_nearest_station_list.append(tabe_station)


In [ ]:
#ジャンルを正しくするためにこれをやる
restaurant_hon_list = []
for tabe_soup in tabe_soup_list:
  tabe_genre = tabe_soup.find('table', class_='c-table--form')
  tabe_genre = tabe_genre.find_all('th')[1].text
  restaurant_hon_list.append(tabe_genre)

#ジャンルをrestaurant_genre_listに格納
restaurant_genre_list = []
for tabe_soup,tabe_hon in zip(tabe_soup_list,restaurant_hon_list):
  if tabe_hon == 'ジャンル':
      tabe_genre = tabe_soup.find('table', class_='c-table')
      tabe_genre = tabe_genre.find_all('td')[1].text
      tabe_genre = tabe_genre.replace('\n','')
      restaurant_genre_list.append(tabe_genre)
  else:
      tabe_genre = tabe_soup.find('table', class_='c-table')
      tabe_genre = tabe_genre.find_all('td')[2].text
      tabe_genre = tabe_genre.replace('\n','')
      restaurant_genre_list.append(tabe_genre)

In [ ]:
#dinner予算をrestaurant_dinner_listに格納
restaurant_dinner_list = []
for tabe_soup in tabe_soup_list:
  tabe_dinner = tabe_soup.find('span', class_='rdheader-budget__price').text
  tabe_dinner = tabe_dinner.replace('\n','')
  restaurant_dinner_list.append(tabe_dinner)

In [ ]:
#lunch予算をrestaurant_lunch_listに格納
restaurant_lunch_list = []
for tabe_soup in tabe_soup_list:
  tabe_lunch = tabe_soup.find_all('span', class_='rdheader-budget__price')[1].text
  tabe_lunch = tabe_lunch.replace('\n','')
  restaurant_lunch_list.append(tabe_lunch)

In [ ]:
#住所をrestaurant_address_listに格納
restaurant_address_list = []
for tabe_soup in tabe_soup_list:
  tabe_address = tabe_soup.find('p',class_='rstinfo-table__address').text
  restaurant_address_list.append(tabe_address)

In [ ]:
#都道府県をrestaurant_prefectures_listに格納
restaurant_prefectures_list = []
for tabe_soup in tabe_soup_list:
  try:
    tabe_prefectures = tabe_soup.find('a',class_='listlink').text
    restaurant_prefectures_list.append(tabe_prefectures)
  except AttributeError:
    restaurant_prefectures_list.append('記載なし')


In [ ]:
"""#市区町村をrestaurant_city_listに格納
restaurant_city_list = []
for tabe_soup in tabe_soup_list:
  tabe_city = tabe_soup.find_all('a',class_='listlink')[1].text
  restaurant_city_list.append(tabe_city)"""
#謎エラー

In [ ]:
"""#地名をrestaurant_place_name_listに格納
restaurant_place_name_list = []
for tabe_soup in tabe_soup_list:
  tabe_place_name = tabe_soup.find_all('a',class_='listlink')[2].text
  restaurant_place_name_list.append(tabe_place_name)"""
  #謎エラーのためやめる

In [ ]:
tuple_element_list = [ [name, hosi, eva,near_station,genre,dinner,lunch,address,prefectures] for name, hosi, eva,near_station,genre,dinner,lunch,address,prefectures in zip (restaurant_name_list, restaurant_hosi_list,restaurant_eva_list,restaurant_nearest_station_list,
                                                                                                           restaurant_genre_list,restaurant_dinner_list,restaurant_lunch_list,restaurant_address_list,restaurant_prefectures_list)]
taberogu_df = pd.DataFrame(tuple_element_list, columns=['店名', '星', '評価数', '最寄駅','ジャンル','予算（夜）','予算(昼）','住所','都道府県'])

In [ ]:
taberogu_df

,店名,星,評価数,最寄駅,ジャンル,予算（夜）,予算(昼）,住所,都道府県
0,日本橋蛎殻町 すぎた,4.71,67306,水天宮前駅,寿司,"￥30,000～￥39,999","￥30,000～￥39,999",東京都中央区日本橋蛎殻町1-33-6 ビューハイツ日本橋 B1F,東京都
1,にい留,4.67,29107,高岳駅,天ぷら,"￥30,000～￥39,999",-,愛知県名古屋市東区泉2-19-11 キャストビル泉 2F,愛知県
2,松川,4.65,40342,六本木一丁目駅,割烹・小料理,"￥50,000～￥59,999","￥50,000～￥59,999",東京都港区赤坂1-11-6 赤坂テラスハウス １階,東京都
3,片折,4.64,25188,北鉄金沢駅,懐石・会席料理,"￥30,000～￥39,999","￥30,000～￥39,999",石川県金沢市並木町3-36,石川県
4,鮨 さいとう,4.63,87503,六本木一丁目駅,寿司,"￥20,000～￥29,999","￥10,000～￥14,999",東京都港区六本木1-4-5 アークヒルズサウスタワー 1Ｆ,東京都
...,...,...,...,...,...,...,...,...,...
95,くろぎ,4.36,28136,大門駅,懐石・会席料理,"￥50,000～￥59,999","￥50,000～￥59,999",東京都港区芝公園1-7-10,東京都
96,鮨匠 のむら,4.36,11348,天文館通駅,寿司,"￥20,000～￥29,999","￥20,000～￥29,999",鹿児島県鹿児島市松原町6-2 松原ハイツ １F,鹿児島県
97,仁修樓,4.36,7359,北大路駅,中華料理,"￥20,000～￥29,999",-,京都府京都市北区紫竹北栗栖町2-12,京都府
98,余市SAGRA,4.36,8710,余市駅,イタリアン、オーベルジュ,-,"￥6,000～￥7,999",北海道余市郡余市町登町987-2,北海道
